### Lab Assignment: Python Warmup and Logfile Analytics

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: August 20, 2023

---

This lab consists of two parts. Part 1 is the Python warmup. Part 2 is the logfile analytics in Spark.  
Answer the questions in this assignment, showing all code and solutions.

**Total points: 15**

---

### Part 1: Python Warmup

1) (1 PT) Rename this notebook to JupyterTutorial_[your_initials], where you will enter your initials in place of [your_initials].

2) (1 PT) In the cell below, enter a list of data science topics you find interesting.  Use the markdown style (you will need to change the style from the Code style).

Computer Vision

3) (1 PT) In the cell below, enter the following Python list:

some_vals = [-1, 6, 12, 34]  

You will use the Code style, run the cell, and print the list.

In [2]:
some_vals = [-1, 6, 12, 34]

print(f"some_vals: {some_vals}")

some_vals: [-1, 6, 12, 34]


4) (1 PT) Use a list comprehension to return a filtered list containing only the values greater than 6.  
Call this list *some_vals_filtered* and print it.

In [3]:
some_vals_filtered = [x for x in some_vals if x > 6]
print(some_vals_filtered)

[12, 34]


Next, a small pandas dataframe is constructed.

In [4]:
import pandas as pd

df = pd.DataFrame({'first_name': ['Andy','Crystal'],
                   'domain_facebook' : [1,1],
                   'domain_foursquare' : [0,0],
                   'age' : [20, 32]})
df

,first_name,domain_facebook,domain_foursquare,age
0,Andy,1,0,20
1,Crystal,1,0,32


5) (1 PT) In the cell below, write a list comprehension that returns the fields names in the dataframe `df` containing the string *domain*.  Run the cell to verify the correct result.

In [5]:
domain_fields = [x for x in df.columns if 'domain' in x]
domain_fields

['domain_facebook', 'domain_foursquare']

6) (1 PT) Use the list comprehension from (5) to index into `df` and show the data for columns containing *domain*

In [6]:
df[domain_fields]

,domain_facebook,domain_foursquare
0,1,0
1,1,0


7) (1 PT) In the cell below, print the *domain_facebook* column

In [30]:
print(df["domain_facebook"])

0    1
1    1
Name: domain_facebook, dtype: int64


8) (1 PT) In the cell below, print the row with index 1.

In [8]:
print(df[1:])

  first_name  domain_facebook  domain_foursquare  age
1    Crystal                1                  0   32


9) (1 PT) Next, you will cube the *age* column of `df` and assign the result to a new column called *agecube*.

Specifically, call the `apply` method with a `lambda function` inside to cube the *age* column.  
Print the dataframe.

In [9]:
df['agecube'] = df['age'].apply(lambda x: x * x * x)
df

,first_name,domain_facebook,domain_foursquare,age,agecube
0,Andy,1,0,20,8000
1,Crystal,1,0,32,32768


10) (1 PT) Given the list of strings below, form one string, placing semicolons between each word.  It should look like this:  

`'the;quick;brown;fox'`

Print the resulting string.

In [10]:
some_list = ['the','quick','brown','fox']

In [31]:
word = ''
for key, value in enumerate(some_list):
    if key == len(some_list) - 1:
        word += value
    else:
        word += value + ';'
    
print(word)

the;quick;brown;fox


### Part 2: Logfile Analytics

Import modules for Spark Session and regex 

In [2]:
from pyspark.sql import SparkSession
import re

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

11) (1 PT) Read in the logfile.txt data

In [13]:
lines = sc.textFile("logfile.txt")
type(lines)

pyspark.rdd.RDD

12) (1 PT) Count the number of rows of data

In [14]:
lines.count()

360

13) (1 PT) Show the first 5 lines

In [15]:
# show some records
lines.take(5)

[' 01 ',
 '03/22 08:51:01 INFO   :.main: *************** RSVP Agent started ***************',
 ' 02 ',
 '03/22 08:51:01 INFO   :...locate_configFile: Specified configuration file: /u/user10/rsvpd1.conf',
 '03/22 08:51:01 INFO   :.main: Using log level 511']

14) (1 PT) Show all lines containing the word WARNING and write code to count them

In [42]:
warning_lines = lines.filter(lambda x: "WARNING" in x)
# return list of first 5 records
warning_lines.take(5)   

['03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.']

In [43]:
wordcounts = words.filter(lambda x: "WARNING" in x) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x,y:x+y) \
                  .map(lambda x:(x[1],x[0])) \
                  .sortByKey(False)
wordcounts.take(5)

[(4, 'WARNING:.....mailslot_create:')]

15) (1 PT) Write a word count program to count the number of each of these log levels:  

terms = 'WARNING|INFO|EVENT|PROTERR|TRACE'

In [45]:
def word_count(word, lines):
    words = lines.flatMap(lambda x: x.split())
    wordcounts = words.filter(lambda x: word in x) \
                      .map(lambda x: (x, 1)) \
                      .reduceByKey(lambda x,y:x+y) \
                      .map(lambda x:(x[1],x[0])) \
                      .sortByKey(False)
    return wordcounts.take(10)

terms = 'WARNING|INFO|EVENT|PROTERR|TRACE'

term = terms.split("|")

# word counts
for word in term:
    print(word_count(word, lines))



[(4, 'WARNING:.....mailslot_create:')]
[(145, 'INFO')]
[(13, 'EVENT')]
[(1, 'PROTERR:.......rsvp_flow_stateMachine:')]
[(119, 'TRACE')]
